<a href="https://colab.research.google.com/github/Buzaabah/livelossplot/blob/master/seq2seq_nl2sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd original\ copy

In [0]:
#Important parckages

%matplotlib inline

import re,string
import numpy as np
import pickle
import spacy
from spacy.symbols import ORTH

en_tok = spacy.load('en')

from collections import Counter,defaultdict
from pathlib import Path


In [10]:
#Reading required data

train_data = open('atis.uw.train.txt').readlines()
train_data[:5]
#len(train_data)

["list all the flights that arrive at general mitchell international from various cities ||| SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport airport_1 , airport_service airport_service_1 , city city_1 WHERE flight_1.to_airport = airport_1.airport_code AND airport_1.airport_code = 'MKE' AND flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code\n",
 "give me the flights leaving denver august ninth coming back to boston ||| SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 , days days_1 , date_day date_day_1 WHERE flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'DENVER' AND ( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'BOSTON' AND flight_1.flight_days = 

In [0]:
test_data = open('atis.uw.test.txt').readlines()
test_data[:5]
len(test_data)

447

In [0]:
dev_data = open('atis.uw.dev.txt').readlines()
dev_data[:5]

len(dev_data)

486

In [0]:
# Combining train, dev and test sets

file_list = ['atis.uw.train.txt', 'atis.uw.dev.txt', 'atis.uw.test.txt']
output_file = Path('Out_file.txt')
for file in file_list:
  output_file.write_text(Path(file).read_text())

In [0]:
# Reading a combined file

data = open('Out_file.txt').readlines()
#data[:5]
len(data)

447

In [0]:
# making a corpus

def make_corpus(corpus_path):
    en,sql = [],[]
    for line in train_data:
        if '\n' in line:
            new_line = line.replace('\n','')
            new_line = new_line.split('|||')
            en.append(new_line[0])
            sql.append(new_line[1])
    return en,sql

In [12]:
#unpacking the corpus

en, sql = make_corpus('atis.uw.train.txt')
en[:2],sql[:2]

(['list all the flights that arrive at general mitchell international from various cities ',
  'give me the flights leaving denver august ninth coming back to boston '],
 [" SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport airport_1 , airport_service airport_service_1 , city city_1 WHERE flight_1.to_airport = airport_1.airport_code AND airport_1.airport_code = 'MKE' AND flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code",
  " SELECT DISTINCT flight_1.flight_id FROM flight flight_1 , airport_service airport_service_1 , city city_1 , airport_service airport_service_2 , city city_2 , days days_1 , date_day date_day_1 WHERE flight_1.from_airport = airport_service_1.airport_code AND airport_service_1.city_code = city_1.city_code AND city_1.city_name = 'DENVER' AND ( flight_1.to_airport = airport_service_2.airport_code AND airport_service_2.city_code = city_2.city_code AND city_2.city_name = 'BOSTON' AND flight_1.flight_

In [0]:
#pre-processing for tokenizing 
# Natural language tokens

def en_tokenizer(text):
    text = text.lower()
    return [t.text for t in en_tok.tokenizer(text)]
  
#SQL tokens
def en_tokenizer(text):
    text = text
    return [t.text for t in en_tok.tokenizer(text)]

In [14]:
# Tokens

en_toks = [en_tokenizer(text) for text in en]
sql_toks = [en_tokenizer(text) for text in sql]
en_toks[:2], sql_toks[:2]

([['list',
   'all',
   'the',
   'flights',
   'that',
   'arrive',
   'at',
   'general',
   'mitchell',
   'international',
   'from',
   'various',
   'cities'],
  ['give',
   'me',
   'the',
   'flights',
   'leaving',
   'denver',
   'august',
   'ninth',
   'coming',
   'back',
   'to',
   'boston']],
 [[' ',
   'SELECT',
   'DISTINCT',
   'flight_1.flight_id',
   'FROM',
   'flight',
   'flight_1',
   ',',
   'airport',
   'airport_1',
   ',',
   'airport_service',
   'airport_service_1',
   ',',
   'city',
   'city_1',
   'WHERE',
   'flight_1.to_airport',
   '=',
   'airport_1.airport_code',
   'AND',
   'airport_1.airport_code',
   '=',
   "'",
   'MKE',
   "'",
   'AND',
   'flight_1.from_airport',
   '=',
   'airport_service_1.airport_code',
   'AND',
   'airport_service_1.city_code',
   '=',
   'city_1.city_code'],
  [' ',
   'SELECT',
   'DISTINCT',
   'flight_1.flight_id',
   'FROM',
   'flight',
   'flight_1',
   ',',
   'airport_service',
   'airport_service_1',
   ',

In [15]:
len(en_toks), len(sql_toks)

(4350, 4350)

In [0]:
# Creating a dictionary/numerical order

def numericalize_tok(tokens, max_vocab=10000, min_freq=0, unk_tok="xxunk", pad_tok="xxpad", bos_tok="xxbos", eos_tok="xxeos"):
    if isinstance(tokens, str):
        raise ValueError("Expected to receive a list of tokens. Received a string instead")
    if isinstance(tokens[0], list):
        tokens = [v for w in tokens for v in w]
    freq = Counter(tokens)
    int2tok = [w for w,z in freq.most_common(max_vocab) if z>min_freq]
    unk_id = 3
    int2tok.insert(0, bos_tok)
    int2tok.insert(1, pad_tok)
    int2tok.insert(2, eos_tok)
    int2tok.insert(unk_id, unk_tok)
    tok2int = defaultdict(lambda:unk_id, {n:k for k,n in enumerate(int2tok)})
    return int2tok, tok2int
  
 

In [0]:
int2sql,sql2int = numericalize_tok(sql_toks)
int2en,en2int = numericalize_tok(en_toks)

In [18]:
len(int2sql), len(int2en)

(688, 859)

In [0]:
ls

In [0]:
# serializing (Saving) a list of tokens for further usage 
# english tokens

int2en_out = open("int2en.pkl","wb")
pickle.dump(int2en, int2en_out)
int2en_out.close()

In [0]:
# serializing (Saving) a list of tokens for further usage 
# SQL tokens

int2sql_out = open("int2sql.pkl","wb")
pickle.dump(int2sql, int2sql_out)
int2sql_out.close()

In [0]:
# Reading a list of tokens for further usage 
# English tokens
  
  with open ('int2en.pkl', 'rb') as fp:
    int2en = pickle.load(fp)
      

In [0]:
# Reading a list of tokens for further usage 
# SQL tokens

with open ('int2sql.pkl', 'rb') as fp:
  int2sql = pickle.load(fp)

In [23]:
type(int2en), type(int2sql)

(list, list)

In [24]:
# Creating indices

sql2int = defaultdict(lambda:3, {n:k for k,n in enumerate(int2sql)})
en2int = defaultdict(lambda:3, {n:k for k,n in enumerate(int2en)})



len(sql2int), len(en2int)

(688, 859)

In [25]:

sql_ids = np.array([[0]+[sql2int[o] for o in sent]+[2] for sent in sql_toks])
en_ids = np.array([[0]+[en2int[o] for o in sent]+[2] for sent in en_toks])
len(sql_ids),len(en_ids), sql_ids[3],en_ids[3]

(4350,
 4350,
 [0,
  25,
  12,
  24,
  41,
  13,
  49,
  51,
  6,
  50,
  54,
  6,
  19,
  22,
  6,
  9,
  23,
  6,
  8,
  17,
  6,
  9,
  33,
  6,
  8,
  30,
  14,
  69,
  4,
  10,
  12,
  148,
  10,
  69,
  11,
  13,
  49,
  51,
  6,
  50,
  54,
  6,
  19,
  22,
  6,
  9,
  23,
  6,
  8,
  17,
  6,
  9,
  33,
  6,
  8,
  30,
  14,
  114,
  4,
  7,
  115,
  7,
  5,
  41,
  4,
  52,
  5,
  53,
  4,
  18,
  5,
  58,
  4,
  7,
  116,
  7,
  5,
  10,
  27,
  4,
  21,
  5,
  20,
  4,
  15,
  5,
  16,
  4,
  7,
  48,
  7,
  5,
  26,
  4,
  31,
  5,
  32,
  4,
  28,
  5,
  29,
  4,
  7,
  59,
  7,
  11,
  11,
  5,
  41,
  4,
  52,
  5,
  53,
  4,
  18,
  5,
  58,
  4,
  7,
  116,
  7,
  5,
  10,
  27,
  4,
  21,
  5,
  20,
  4,
  15,
  5,
  16,
  4,
  7,
  48,
  7,
  5,
  26,
  4,
  31,
  5,
  32,
  4,
  28,
  5,
  29,
  4,
  7,
  59,
  7,
  11,
  2],
 [0, 9, 23, 7, 268, 169, 67, 70, 40, 5, 12, 4, 20, 8, 68, 31, 2])

In [0]:
# we set seed for reproducibility/splitting train and test

np.random.seed(42)

In [27]:
#Padding for ebatch normalization

import keras
from keras.preprocessing.sequence import pad_sequences

englen_90 = int(np.percentile([len(o) for o in en_ids], 99))
sqllen_90 = int(np.percentile([len(o) for o in sql_ids], 99))
englen_90, sqllen_90

Using TensorFlow backend.


(27, 254)

In [0]:
sql_ids = pad_sequences(sql_ids, maxlen=254, dtype='int32', padding='post', truncating='post', value=1)
en_ids = pad_sequences(en_ids, maxlen=27, dtype='int32', padding='post', truncating='post', value=1)

In [29]:
trn_keep = np.random.rand(len(en_ids))>0.1
en_trn,sql_trn = en_ids[trn_keep],sql_ids[trn_keep]
en_val,sql_val = en_ids[~trn_keep],sql_ids[~trn_keep]
len(en_trn),len(en_val)

(3886, 464)

In [0]:
# Using pytorch framework to train seq2seq

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd.variable import Variable
from torch.utils.data import Dataset,DataLoader


In [0]:
from numpy import array as A

In [0]:
#Creating a seq2seq

class seq2seqDataset(Dataset):
  def __init__(self, x, y): self.x,self.y = x,y
  def __getitem__(self, idx): return A(self.x[idx]), A(self.y[idx])
  def __len__(self): return len(self.x)
  
  

In [0]:
# Training sequence

trn_ds = seq2seqDataset(en_trn,sql_trn)
val_ds = seq2seqDataset(en_val,sql_val)

bs = 120 #batch size

# Loading training dataset

trn_dl = DataLoader(trn_ds,batch_size=bs,shuffle=True)
val_dl = DataLoader(val_ds,batch_size=bs)

In [34]:
x, y = next(iter(val_dl))
x.size(), y.size()



(torch.Size([120, 27]), torch.Size([120, 254]))

In [0]:
## load fasttext vectors
#code from here:https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md
import io
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    header = fin.readline().split()
    n, d = int(header[0]), int(header[1])
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(tokens[1:], dtype=float)
    return data, int(n), int(d)

In [0]:
# get word vectors
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
#!unzip wiki-news-300d-1M.vec.zip
#!mv wiki-news-300d-1M.vec data/


In [8]:
ls


academic.txt          geography.uw.test.txt   scholar.uw.dev.txt
atis-notes.md         geography.uw.train.txt  scholar.uw.test.txt
atis.uw.dev.txt       imdb.txt                scholar.uw.train.txt
atis.uw.test.txt      int2en.pkl              wiki-news-300d-1M.vec
atis.uw.train.txt     int2sql.pkl             wiki-news-300d-1M.vec.zip
cc.ja.300.vec.gz      Out_file.txt            yelp.txt
geography-db.txt      README.md
geography.uw.dev.txt  scholar-notes.md


In [0]:
# Loading vectors

en_vecs,_,dim_en_vec = load_vectors('wiki-news-300d-1M.vec')
sql_vecs,_,dim_sql_vec = load_vectors('wiki-news-300d-1M.vec')

In [0]:
# Creating embbeddings

def create_emb(vecs, itos, em_sz):
    emb = nn.Embedding(len(itos), em_sz, padding_idx=1)
    if vecs is None: return emb
    wgts = emb.weight.data
    miss = []
    for i,w in enumerate(itos):
        try: wgts[i] = torch.from_numpy(vecs[w])
        except: miss.append(w)
    print('Number of unknowns in data: {}'.format(len(miss)))
    return emb
    

In [0]:
def V(tensor,req_grad=True):
    if torch.cuda.is_available():return Variable(tensor.cuda())
    else: return Variable(tensor)

In [0]:
# Seq2Seq Architecture
# a multilayered Long Short-Term Memory (LSTM) to map the input sequence to a vector of a fixed dimensionality, 
# and then another deep LSTM to decode the target sequence from the vector.

# sl: sequence length, bs: batch size, nh: number of hideen units

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self,int2en,int2sql,em_sz,sql_vecs=None,en_vecs=None,nh=128,out_sl=27,dropf=1,nl=2):
        super().__init__()
        #encoder
        self.nl,self.nh,self.em_sz,self.out_sl = nl,nh,em_sz,out_sl
        self.emb_enc = create_emb(en_vecs,int2en,em_sz)
        self.emb_drop = nn.Dropout(0.15*dropf)
        self.encoder = nn.GRU(em_sz,nh,num_layers=nl,dropout=0.25*dropf, bidirectional=True)
        #decoder
        self.emb_dec = create_emb(sql_vecs,int2sql,em_sz)
        self.decoder = nn.GRU(em_sz,nh*2,num_layers=nl,dropout=0.25*dropf)
        self.out_drop = nn.Dropout(0.35*dropf)
        self.out = nn.Linear(nh*2,len(int2sql))
    
    def forward(self,inp,y=None):
        sl, bs = inp.size()
        emb_in = self.emb_drop(self.emb_enc(inp))
        h_n = self.initHidden(bs)
        enc_out, h_n = self.encoder(emb_in,h_n)
        h_n = h_n.view(2,2,bs,-1).permute(0,2,1,3).contiguous().view(self.nl,bs,-1)
        
        dec_inp = V(torch.zeros(bs).long())
        res = []
        for i in range(self.out_sl):
            dec_emb = self.emb_dec(dec_inp)
            outp,h_n = self.decoder(dec_emb.unsqueeze(0),h_n)
            outp = F.log_softmax(self.out(self.out_drop(outp[0])),dim=-1)
            res.append(outp)
            dec_inp = outp.data.max(1)[1]
            if (dec_inp==1).all(): break
        return torch.stack(res)
        
    def initHidden(self,bs):
        return V(torch.zeros([self.nl*2,bs,self.nh]))

In [0]:
# Reversing the order of the words (Bidirectional encoder) in all source sentences (butnot target sentences) improves the LSTM’s 
# performance, because doing so introduced many short term dependencies between the source and the targetsentence which 
# makes the optimization problem easier.

# Input and output sequences may not directly map to each other so preserving information from both passes of the input sequence 
# will help learn how tokens relate to each other. For example in a translation task, subject and object can be in opposite positions 
# depending on the language structure.

In [43]:
seq2seq = Seq2Seq(int2en,int2sql,300,en_vecs=None,sql_vecs=None)
seq2seq.cuda()

Seq2Seq(
  (emb_enc): Embedding(859, 300, padding_idx=1)
  (emb_drop): Dropout(p=0.15)
  (encoder): GRU(300, 128, num_layers=2, dropout=0.25, bidirectional=True)
  (emb_dec): Embedding(688, 300, padding_idx=1)
  (decoder): GRU(300, 256, num_layers=2, dropout=0.25)
  (out_drop): Dropout(p=0.35)
  (out): Linear(in_features=256, out_features=688, bias=True)
)

In [44]:
out = seq2seq(V(x.transpose(1,0).long()))
out.size()

torch.Size([27, 120, 688])

In [0]:
#Loss Function: 
# Using the Cross Entropy Loss as we are trying to classify out the correct words. Cross entropy loss can be simplified to:

#     cross_entropy = sum(-log(y_pred) for y_pred in y_preds)

# y_pred is the likelihood of the target class predicted by the model. This is a good loss function for classification
# because if the likelihood of the correct class is low, the loss value goes up and if it is high, the loss value goes down.

In [0]:
def seq2seq_loss(input, target):
    sl,bs = target.size()
    sl_in,bs_in,nc = input.size()
    if sl>sl_in: input = F.pad(input, (0,0,0,0,0,sl-sl_in))
    input = input[:sl]
    return F.cross_entropy(input.view(-1,nc), target.view(-1))

In [47]:
seq2seq_loss(out,V(y.transpose(1,0).long()))

tensor(6.5355, device='cuda:0', grad_fn=<NllLossBackward>)

In [0]:
def step(x, y, epoch, m, crit, opt, clip=None):
    output = m(x, y)
    if isinstance(output,tuple): output = output[0]
    opt.zero_grad()
    loss = crit(output, y)
    loss.backward()
    if clip:
        nn.utils.clip_grad_norm_(m.parameters(), clip)
    opt.step()
    return loss.data.item()

In [0]:
from tqdm import tqdm

In [0]:
# In pytorch we have to write a training function

def train(trn_dl,val_dl,model,crit,opt,epochs= 20,clip=None):
    for epoch in range(epochs):
        loss_val = loss_trn = 0
        with tqdm(total=len(trn_dl)) as pbar:
            model.train()
            for i, ds in enumerate(trn_dl):
                x, y = ds
                #if isinstance(x,tuple): x = x[0]
                x, y = x.transpose(1,0), y.transpose(1,0)
                loss = step(V(x.long()),V(y.long()),epoch,model,crit,opt)
                loss_trn += loss
                pbar.update()
        model.eval()
        for i, ds in enumerate(val_dl):
            with torch.no_grad():
                x, y = ds
                #if isinstance(x,tuple): x = x[0]
                x, y = x.transpose(1,0), y.transpose(1,0)
                out = model(V(x.long()))
                if isinstance(out,tuple): out = out[0]
                loss_val+= crit(out, V(y.long()))
                #loss_val +=loss
        print(f'Epoch: {epoch} trn loss: {loss_trn/len(trn_dl)} val loss: {loss_val/len(val_dl)}')

In [0]:
from torch import optim

In [0]:
opt = optim.Adam(seq2seq.parameters(),lr=3e-3,betas=(0.7,0.8))

In [53]:
train(trn_dl,val_dl,seq2seq,seq2seq_loss,opt,epochs=20)

  6%|▌         | 2/33 [00:00<00:01, 17.02it/s]

Epoch: 0 trn loss: 6.142410336118756 val loss: 6.063150882720947


  6%|▌         | 2/33 [00:00<00:01, 15.99it/s]

Epoch: 1 trn loss: 6.021258296388568 val loss: 5.971856117248535


  6%|▌         | 2/33 [00:00<00:01, 19.11it/s]

...

Epoch: 17 trn loss: 5.87238733696215 val loss: 5.893012046813965


  6%|▌         | 2/33 [00:00<00:01, 18.24it/s]

Epoch: 18 trn loss: 5.870857917901241 val loss: 5.895094871520996


100%|██████████| 33/33 [00:01<00:00, 18.58it/s]


Epoch: 19 trn loss: 5.8691605221141465 val loss: 5.893664836883545


In [0]:
# Printing the output

def produce_out(val_dl, model,int2en,int2sql,interval=(20,30)):
    model.eval()
    x,y = next(iter(val_dl))
    x, y = x.transpose(1,0), y.transpose(1,0)
    probs = seq2seq(V(x.long()))
    if isinstance(probs,tuple): probs = probs[0] 
    preds = A(probs.max(2)[1].cpu())
    for i in range(interval[0],interval[1]):
        print(' '.join([int2en[o] for o in x[:,i] if o not in [0,1,2]]))
        print(''.join([int2sql[o] for o in y[:,i] if o not in [0,1,2]]))
        print(''.join([int2sql[o] for o in preds[:,i] if o not in [0,1,2]]))
        print()

In [55]:
produce_out(trn_dl,seq2seq,int2en,int2sql)

please show me all airlines with sunday flights from philadelphia to denver
 SELECTDISTINCTairline_1.airline_codeFROMairlineairline_1,flightflight_1,airport_serviceairport_service_1,citycity_1,airport_serviceairport_service_2,citycity_2,daysdays_1WHEREairline_1.airline_code=flight_1.airline_codeANDflight_1.from_airport=airport_service_1.airport_codeANDairport_service_1.city_code=city_1.city_codeANDcity_1.city_name='PHILADELPHIA'AND(flight_1.to_airport=airport_service_2.airport_codeANDairport_service_2.city_code=city_2.city_codeANDcity_2.city_name='DENVER'ANDflight_1.flight_days=days_1.days_codeANDdays_1.day_name='SUNDAY')
 SELECTDISTINCTairline_1.airline_codeFROMairlineairline_1,flightflight_1,airport_serviceairport_service_1,citycity_1,airport_serviceairport_service_2,citycity_2,daysdays_1,

show me flights from boston to philadelphia on a monday
 SELECTDISTINCTflight_1.flight_idFROMflightflight_1,airport_serviceairport_service_1,citycity_1,airport_serviceairport_service_2,citycity_2,

In [0]:
torch.save(seq2seq.state_dict(),open('translate_seq2seq.pth','wb'))

In [0]:
seq2seq.load_state_dict(torch.load('translate_seq2seq.pth', map_location=lambda storage, loc: storage))